<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/NorthwesternHeader.png?raw=1">

## MSDS458 Research Assignment 3 - Part 01

## Analyze AG_NEWS_SUBSET Data <br>

AG is a collection of more than 1 million news articles. News articles have been gathered from more than 2000 news sources by ComeToMyHead in more than 1 year of activity. ComeToMyHead is an academic news search engine which has been running since July, 2004. The dataset is provided by the academic comunity for research purposes in data mining (clustering, classification, etc), information retrieval (ranking, search, etc), xml, data compression, data streaming, and any other non-commercial activity.<br>

For more information, please refer to the link http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html<br>


The AG's news topic classification dataset is constructed by choosing 4 largest classes (**World**, **Sports**, **Business**, and **Sci/Tech**) from the original corpus. Each class contains 30,000 training samples and 1,900 testing samples. The total number of training samples is 120,000 and testing 7,600.<br>

Homepage: https://arxiv.org/abs/1509.01626<br>

Source code: tfds.text.AGNewsSubset

Versions:

1.0.0 (default): No release notes.
Download size: 11.24 MiB

Dataset size: 35.79 MiB

## References
1. Deep Learning with Python, Francois Chollet (https://learning.oreilly.com/library/view/deep-learning-with/9781617296864/)
 * Chapter 10: Deep learning for time series
 * Chapter 11: Deep learning for text
2. Deep Learning A Visual Approach, Andrew Glassner (https://learning.oreilly.com/library/view/deep-learning/9781098129019/)
 * Chapter 19: Recurrent Neural Networks
 * Chapter 20: Attention and Transformers

# Deep learning for text

## Natural-language processing: The bird's eye view

## Preparing text data

<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/11-01.png?raw=1">

## Import Packages

In [2]:
from packaging import version

import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from keras import layers
from keras.layers import TextVectorization

## Verify TensorFlow version and Keras version

In [3]:
print("This notebook requires TensorFlow 2.0 or above")
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >=2

This notebook requires TensorFlow 2.0 or above
TensorFlow version:  2.15.0


In [4]:
print("Keras version: ", keras.__version__)

Keras version:  2.15.0


## Mount Google Drive to Colab environment

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/MSDS_458/Assignment 3/data/
gCollab = True

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/MSDS_458/Assignment 3/data


## Load AG_NEWS_SUBSET News Articles Dataset

In [7]:
# register  ag_news_subset so that tfds.load doesn't generate a checksum (mismatch) error
!python -m tensorflow_datasets.scripts.download_and_prepare --register_checksums --datasets=ag_news_subset

dataset, info = tfds.load('ag_news_subset', with_info=True,  split=['train[:114000]','train[114000:]', 'test'],
                          batch_size = 32, as_supervised=True)
train_ds, val_ds, test_ds = dataset

W0508 01:01:55.393962 134767467241472 download_and_prepare.py:46] ***`tfds build` should be used instead of `download_and_prepare`.***
INFO[build.py]: Loading dataset ag_news_subset from imports: tensorflow_datasets.datasets.ag_news_subset.ag_news_subset_dataset_builder
2024-05-08 01:01:55.862055: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 01:01:55.862107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 01:01:55.863764: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 01:01:57.579372: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

## Display The Number of Batches

In [8]:
len(train_ds), len(val_ds), len(test_ds)

(3563, 188, 238)

## Displaying The Shapes and Dtypes of the First Batch

In [9]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print()
    print("inputs.dtype:", inputs.dtype)
    print()
    print("targets.shape:", targets.shape)
    print()
    print("targets.dtype:", targets.dtype)
    print()
    print("inputs[0]:", inputs[0])
    print()
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)

inputs.dtype: <dtype: 'string'>

targets.shape: (32,)

targets.dtype: <dtype: 'int64'>

inputs[0]: tf.Tensor(b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.', shape=(), dtype=string)

targets[0]: tf.Tensor(3, shape=(), dtype=int64)


## Processing words as a set: The bag-of-words approach

The simplest way to encode a piece of text for processing by a machine learning model is to discard order and treat it as a set (a “bag”) of tokens.

## Single words (unigrams) with binary encoding

The main advantage of this encoding is that you can represent an entire text as a single vector, where each entry is a presence indicator for a given word.

## Preprocessing Datasets TextVectorization Layer

<div class="alert alert-block alert-success">
    <b>tf.keras.layers.TextVectorization</b><br>
    https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization
    </div>

In [11]:
text_vectorization = TextVectorization(
    max_tokens=1000,
    output_mode="multi_hot")

In [12]:
text_only_train_ds = train_ds.map(lambda x, y: x)

In [13]:
for text in text_only_train_ds:
    print(f"Get first batch of {text.shape[0]} news articles.\n")
    print(f"Here is the first news article:\n\n{text[0]}.")
    break

Get first batch of 32 news articles.

Here is the first news article:

b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'.


## Adapt Method - Standardize Text

In [14]:
text_vectorization.adapt(text_only_train_ds)

In [15]:
binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

## Inspecting Output Binary Unigram Dataset

In [16]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print()
    print("inputs.dtype:", inputs.dtype)
    print()
    print("targets.shape:", targets.shape)
    print()
    print("targets.dtype:", targets.dtype)
    print()
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 1000)

inputs.dtype: <dtype: 'float32'>

targets.shape: (32,)

targets.dtype: <dtype: 'int64'>

targets[0]: tf.Tensor(3, shape=(), dtype=int64)


## Model Function

In [17]:
def get_model(max_tokens=1000, hidden_dim=16):
    inputs = tf.keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(4, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss='SparseCategoricalCrossentropy',
                  metrics=["accuracy"])
    return model

## Build Binary Unigram Model

In [18]:
model_Unigram = get_model()
model_Unigram.summary()
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("binary_1gram.keras",save_best_only=True)
    ,tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
]

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000)]            0         
                                                                 
 dense (Dense)               (None, 16)                16016     
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 68        
                                                                 
Total params: 16084 (62.83 KB)
Trainable params: 16084 (62.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model_Unigram.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=200,
          callbacks=callbacks)
model_Unigram = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model_Unigram.evaluate(binary_1gram_test_ds)[1]:.3f}")

Epoch 1/200
3563/3563 [==============================] - 15s 4ms/step - loss: 0.6801 - accuracy: 0.7555 - val_loss: 0.4678 - val_accuracy: 0.8423
Epoch 2/200
3563/3563 [==============================] - 10s 3ms/step - loss: 0.5947 - accuracy: 0.7987 - val_loss: 0.4770 - val_accuracy: 0.8398
Epoch 3/200
3563/3563 [==============================] - 10s 3ms/step - loss: 0.6030 - accuracy: 0.7950 - val_loss: 0.4813 - val_accuracy: 0.8382
Epoch 4/200
3563/3563 [==============================] - 10s 3ms/step - loss: 0.6064 - accuracy: 0.7929 - val_loss: 0.4818 - val_accuracy: 0.8375
Epoch 5/200
3563/3563 [==============================] - 9s 3ms/step - loss: 0.6038 - accuracy: 0.7934 - val_loss: 0.4786 - val_accuracy: 0.8380
Epoch 6/200
238/238 [==============================] - 1s 3ms/step - loss: 0.4918 - accuracy: 0.8376
Test acc: 0.838


We call `cache()` on the datasets to cache them in memory: this way, we will only do the preprocessing once, during the first epoch, and we’ll reuse the preprocessed texts for the following epochs. This can only be done if the data is small enough to fit in memory.

## Bigrams With Binary Encoding

Of course, discarding word order is very reductive, because even atomic concepts can be expressed via multiple words: the term “United States” conveys a concept that is quite distinct from the meaning of the words “states” and “united” taken separately.

With bigrams, the sentence “`the cat sat on the mat.`” becomes

`{"the", "the cat", "cat", "cat sat", "sat",
 "sat on", "on", "on the", "the mat", "mat"}`

<div class="alert alert-block alert-success">
    <b>tf.keras.layers.TextVectorization</b><br>
    https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization
    </div>

## Configuring the `TextVectorization` layer to return Bigrams

The TextVectorization layer can be configured to return arbitrary N-grams: bigrams, trigrams, etc. Just pass an `ngrams=N` argument as in the following listing.

In [20]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=1000,
    output_mode="multi_hot",
)

## Build Binary Bigram Model

In [21]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model_Bigram = get_model()
model_Bigram.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1000)]            0         
                                                                 
 dense_2 (Dense)             (None, 16)                16016     
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 4)                 68        
                                                                 
Total params: 16084 (62.83 KB)
Trainable params: 16084 (62.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
callbacks = [
     tf.keras.callbacks.ModelCheckpoint("binary_2gram.keras",save_best_only=True)
    ,tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
]

model_Bigram.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=200,
          callbacks=callbacks)
model_Bigram = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model_Bigram.evaluate(binary_2gram_test_ds)[1]:.3f}")

Epoch 1/200
3563/3563 [==============================] - 15s 4ms/step - loss: 0.7056 - accuracy: 0.7435 - val_loss: 0.4854 - val_accuracy: 0.8370
Epoch 2/200
3563/3563 [==============================] - 9s 3ms/step - loss: 0.6234 - accuracy: 0.7875 - val_loss: 0.4965 - val_accuracy: 0.8327
Epoch 3/200
3563/3563 [==============================] - 10s 3ms/step - loss: 0.6255 - accuracy: 0.7866 - val_loss: 0.4973 - val_accuracy: 0.8303
Epoch 4/200
3563/3563 [==============================] - 10s 3ms/step - loss: 0.6212 - accuracy: 0.7866 - val_loss: 0.4941 - val_accuracy: 0.8308
Epoch 5/200
3563/3563 [==============================] - 9s 3ms/step - loss: 0.6151 - accuracy: 0.7875 - val_loss: 0.4905 - val_accuracy: 0.8318
Epoch 6/200
238/238 [==============================] - 1s 2ms/step - loss: 0.5069 - accuracy: 0.8270
Test acc: 0.827


## Bigrams with TF-IDF Encoding

You can also add a bit more information to this representation by counting how many times each word or N-gram occurs, that is to say, by taking the histogram of the words over the text:

```{"the": 2, "the cat": 1, "cat": 1, "cat sat": 1, "sat": 1,
 "sat on": 1, "on": 1, "on the": 1, "the mat: 1", "mat": 1}```

## Understanding TF-IDF normalization
The more a given term appears in a document, the more important that term is for understanding what the document is about. At the same time, the frequency at which the term appears across all documents in your dataset matters too: terms that appear in almost every document (like “the” or “a”) aren’t particularly informative,

`TF-IDF` is a metric that fuses these two ideas. It weights a given term by taking “term frequency,” how many times the term appears in the current document, and dividing it by a measure of “document frequency,” which estimates how often the term comes up across the dataset.

```python
def tfidf(term, document, dataset):
    term_freq = document.count(term)
    doc_freq = math.log(sum(doc.count(term) for doc in dataset) + 1)
    return term_freq / doc_freq
```

## Configure `TextVectorization` Layer To Return Token Counts

In [23]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=1000,
    output_mode="count"
)

## Configuring `TextVectorization` To Return TF-IDF-weighted Outputs

In [24]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=1000,
    output_mode="tf_idf",
)

## Build TF-IDF Bigram Model

In [25]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model_tfidf = get_model()
model_tfidf.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1000)]            0         
                                                                 
 dense_4 (Dense)             (None, 16)                16016     
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 4)                 68        
                                                                 
Total params: 16084 (62.83 KB)
Trainable params: 16084 (62.83 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",save_best_only=True)
   ,tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
]

model_tfidf.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=200,
          callbacks=callbacks)
model_tfidf = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model_tfidf.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Epoch 1/200
3563/3563 [==============================] - 13s 4ms/step - loss: 0.9047 - accuracy: 0.6195 - val_loss: 0.5654 - val_accuracy: 0.8157
Epoch 2/200
3563/3563 [==============================] - 9s 3ms/step - loss: 0.7942 - accuracy: 0.6902 - val_loss: 0.5533 - val_accuracy: 0.8182
Epoch 3/200
3563/3563 [==============================] - 10s 3ms/step - loss: 0.7662 - accuracy: 0.7071 - val_loss: 0.5520 - val_accuracy: 0.8135
Epoch 4/200
3563/3563 [==============================] - 12s 3ms/step - loss: 0.7626 - accuracy: 0.7078 - val_loss: 0.5384 - val_accuracy: 0.8178
Epoch 5/200
3563/3563 [==============================] - 10s 3ms/step - loss: 0.7522 - accuracy: 0.7123 - val_loss: 0.5391 - val_accuracy: 0.8218
Epoch 6/200
3563/3563 [==============================] - 10s 3ms/step - loss: 0.7544 - accuracy: 0.7116 - val_loss: 0.5344 - val_accuracy: 0.8285
Epoch 7/200
3563/3563 [==============================] - 10s 3ms/step - loss: 0.7548 - accuracy: 0.7119 - val_loss: 0.5410 - 

In [27]:
inputs = tf.keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model_tfidf(processed_inputs)
inference_model = tf.keras.Model(inputs, outputs)

In [28]:
inference_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_4 (Text  (None, 1000)              1         
 Vectorization)                                                  
                                                                 
 model_2 (Functional)        (None, 4)                 16084     
                                                                 
Total params: 16085 (62.84 KB)
Trainable params: 16084 (62.83 KB)
Non-trainable params: 1 (8.00 Byte)
_________________________________________________________________


In [29]:
raw_text_data = tf.convert_to_tensor(
    [["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"{predictions.numpy()[0][0] * 100:.2f} percent positive")
predictions.numpy()[0]

20.91 percent positive


array([0.20910475, 0.17504847, 0.22393508, 0.39191172], dtype=float32)

In [30]:
raw_text_data = tf.convert_to_tensor([['''
ATLANTA -- Atlanta Braves shortstop Rafael Furcal has had his first court appearance
after being arrested on charges of driving under the influence.'
''']])
predictions = inference_model(raw_text_data)
print(f"{predictions.numpy()[0][0] * 100:.2f} percent positive")

48.95 percent positive


In [31]:
predictions.numpy()

array([[0.48953596, 0.27454665, 0.09198792, 0.14392948]], dtype=float32)